# Homework 6: Mixed effects models

This homework assignment is designed to give you practice fitting and interpreting mixed effects models. 

We will be using the **LexicalData.csv** and **Items.csv** files from the *Homework/lexDat* folder in the class GitHub repository again. 

This data is a subset of the [English Lexicon Project database](https://elexicon.wustl.edu/). It provides the reaction times (in milliseconds) of many subjects as they are presented with letter strings and asked to decide, as quickly and as accurately as possible, whether the letter string is a word or not. The **Items.csv** provides characteristics of the words used, namely frequency (how common is this word?) and length (how many letters?). Unlike in the previous homework, there isn't any missing data in the **LexicalData.csv** file. 

*Data courtesy of Balota, D.A., Yap, M.J., Cortese, M.J., Hutchison, K.A., Kessler, B., Loftis, B., Neely, J.H., Nelson, D.L., Simpson, G.B., & Treiman, R. (2007). The English Lexicon Project. Behavior Research Methods, 39, 445-459.*

---
## Loading and formatting the data (1 point)

Load in data from the **LexicalData.csv** and **Items.csv** files. As in the previous homeworks, remove the commas from the reaction times and convert them from strings to numbers. Use `left_join` to add word frequencies from **Items** to **LexicalData**. 

*Note: the `Freq_HAL` variable in **Items.csv** has a similar formatting issue, using string values with commas. We're not going to worry about fixing this since we're only using `Log_Freq_HAL`, which is the natural log transformation of `Freq_HAL`, in this homework.*

In [1]:
library(tidyverse)

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.1.0     ✔ dplyr   1.0.5
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [2]:
system("gdown --id 1q0toKETEbZf71Vb1e_iUKB1qplNFghCo") 
lexical_data = read.csv("LexicalData.csv") 

system("gdown --id 12Cx9lvoPb8AeGUWnJngofu4AXevxk45U")
items_data = read.csv("Items.csv")

head(lexical_data)
head(items_data)

comb_data <- left_join(lexical_data %>% 
                                  mutate(D_RT_new = gsub(',', '', D_RT) %>% 
                                  as.numeric()) %>% 
                                  filter(!is.na(D_RT_new),
                                         D_RT_new >= 0) %>%
                                  select(!D_RT), 
                       items_data %>% 
                                  filter(!is.na(Length),
                                         !is.na(Log_Freq_HAL)) %>%
                                  select(Length, Log_Freq_HAL, Word),
                       by = c("D_Word" = "Word"))
head(comb_data)


,Sub_ID,Trial,Type,D_RT,D_Word,Outlier,D_Zscore
,<int>,<int>,<int>,<chr>,<chr>,<chr>,<dbl>
1,157,1,1,710,browse,false,-0.437
2,67,1,1,"1,094",refrigerant,false,0.825
3,120,1,1,587,gaining,false,-0.645
4,21,1,1,984,cheerless,false,0.025
5,236,1,1,577,pattered,false,-0.763
6,236,2,1,715,conjures,false,-0.364


,Occurrences,Word,Length,Freq_HAL,Log_Freq_HAL
,<int>,<chr>,<int>,<chr>,<dbl>
1,1,synergistic,11,284,5.649
2,1,synonymous,10,951,6.858
3,1,syntactical,11,114,4.736
4,1,synthesis,9,"6,742",8.816
5,1,synthesized,11,"2,709",7.904
6,1,synthesizer,11,"1,390",7.237


,Sub_ID,Trial,Type,D_Word,Outlier,D_Zscore,D_RT_new,Length,Log_Freq_HAL
,<int>,<int>,<int>,<chr>,<chr>,<dbl>,<dbl>,<int>,<dbl>
1,157,1,1,browse,false,-0.437,710,6,8.856
2,67,1,1,refrigerant,false,0.825,1094,11,4.644
3,120,1,1,gaining,false,-0.645,587,7,8.304
4,21,1,1,cheerless,false,0.025,984,9,2.639
5,236,1,1,pattered,false,-0.763,577,8,1.386
6,236,2,1,conjures,false,-0.364,715,8,5.268


# Model fitting (4 points)

First, fit a linear model with `Log_Freq_HAL` and `Length` as predictors, and `D_RT` as the output. Include an interaction term. Use `summary()` to look at the model output. 

In [5]:
comb_data.lin <- lm(D_RT_new ~ Log_Freq_HAL * Length, data = comb_data)
comb_data.lin.summ <- summary(comb_data.lin)
comb_data.lin.summ


Call:
lm(formula = D_RT_new ~ Log_Freq_HAL * Length, data = comb_data)

Residuals:
     Min       1Q   Median       3Q      Max 
-1118.01  -205.23   -86.95    90.77  3147.07 

Coefficients:
                    Estimate Std. Error t value Pr(>|t|)    
(Intercept)         610.1903    14.6678  41.601  < 2e-16 ***
Log_Freq_HAL         -6.0239     1.9678  -3.061  0.00221 ** 
Length               47.7531     1.6368  29.175  < 2e-16 ***
Log_Freq_HAL:Length  -2.9421     0.2348 -12.528  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 359.1 on 62606 degrees of freedom
Multiple R-squared:  0.09473,	Adjusted R-squared:  0.09469 
F-statistic:  2184 on 3 and 62606 DF,  p-value: < 2.2e-16


Now, install `lme4` using `install.packages()` and then load the library. 

In [6]:
install.packages("lme4")
library(lme4)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘minqa’, ‘nloptr’, ‘statmod’, ‘RcppEigen’


Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack




Now fit a mixed effects model that includes the same predictors as the linear model above, as well as random intercepts for `Sub_ID` (i.e., cases where subject ID shifts the RT mean). Use `summary()` to look at the model output. 

In [7]:
comb_data.mix <- lmer(D_RT_new ~ Log_Freq_HAL * Length + (1 | Sub_ID), data = comb_data)
comb_data.mix.summ <- summary(comb_data.mix)
comb_data.mix.summ

Linear mixed model fit by REML ['lmerMod']
Formula: D_RT_new ~ Log_Freq_HAL * Length + (1 | Sub_ID)
   Data: comb_data

REML criterion at convergence: 888235.6

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.5058 -0.5472 -0.1568  0.3103 10.7381 

Random effects:
 Groups   Name        Variance Std.Dev.
 Sub_ID   (Intercept) 46333    215.3   
 Residual             82978    288.1   
Number of obs: 62610, groups:  Sub_ID, 299

Fixed effects:
                    Estimate Std. Error t value
(Intercept)         616.8445    17.1522  35.963
Log_Freq_HAL         -7.4374     1.5830  -4.698
Length               47.7477     1.3162  36.277
Log_Freq_HAL:Length  -2.8778     0.1888 -15.239

Correlation of Fixed Effects:
            (Intr) Lg_F_HAL Length
Log_Frq_HAL -0.645                
Length      -0.656  0.917         
Lg_Fr_HAL:L  0.582 -0.942   -0.923

# Model assessment (4 points)

Compare the three t-values for the fixed effects and the mixed effects models. How do they differ, and why? 

> In general, the t-value for each of the terms in the linear model is calculated by dividing the beta estimate for the term by the standard error. When we added `subject ID` as a random variable to the mixed effects model, we are partioning the variance or error of each of the terms differently because we are calculating and parsing out the contribution of the individual or subject ID specific variance from each of the terms in our model. We see the most straight forward evidence of this when comparing the standard errors for each term in our two models. In each case (`length`, `log freq`, and their `interaction`) the standard error of the terms in the mixed effects model are smaller than the fixed effects model. 
- Length
  - Mixed effects       
    - estimate: 47.7477 ... std. error: 1.3162 ... t-statistic: 36.277
  - Fixed effects              
    - estimate: 47.7531 ... std. error: 1.6368 ... t-statistic: 29.175

> When we parse out the individual subject variance from the `length` term, the t-statistic increases compared to the fixed effects t-value. Because the standard error is lower and the beta estimate is largely unaffected by taking the random effects on `length` into account, the t-statistic is higher in the mixed effects model. $\frac{similar \beta}{\text{smaller SE}} > \frac{similar \beta}{\text{bigger SE}}$
- Log frequency:
  - Mixed effects       
    - estimate: -7.4374 ... std. error: 1.5830 ... t-statistic: -4.698
  - Fixed effects              
    - estimate: -6.0239 ... std. error: 1.9678 ... t-statistic: -3.061

> When we parse out the individual subject variance from the `log freq` term, the t-statistic decreases compared to the fixed effects t-value. The standard error is lower and the beta estimate is also lower when taking the random effects on `log freq` into account. Thus the t-statistic is lower in the mixed effects model because removing the random effect of subject ID on `log freq` made the estimate even lower. $\frac{\text{more negative } \beta}{\text{smaller SE}} < \frac{\text{less negative }\beta}{\text{bigger SE}}$
- Length and log frequency interaction:
  - Mixed effects       
    - estimate: -2.8778 ... std. error: 0.1888 ... t-statistic: -15.239
  - Fixed effects
    - estimate: -2.9421 ... std. error: 0.2348 ... t-statistic: -12.528

> When we parse out the individual subject variance from the `interaction` term, the t-statistic decreases compared to the fixed effects t-value. The standard error is lower and the beta estimate is similar when taking the random effects on `log freq` into account. Thus the t-statistic is lower in the mixed effects model because the ratio of the estimate to the standard error is higher (in the negative direction). $\frac{\text{similar negative } \beta}{\text{smaller SE}} < \frac{\text{similar negative }\beta}{\text{bigger SE}}$





Use the Aikeke Information Criterion (AIC) to compare these two models. Which one is better? 

In [9]:
ic = AIC(comb_data.lin, comb_data.mix)
ic
diff(ic$AIC)

,df,AIC
,<dbl>,<dbl>
comb_data.lin,5,914436.4
comb_data.mix,6,888247.6


[1] -26188.82

> The mixed effects model is better choice to model the data because it has the lowest AIC. Because the difference between the fixed effects and mixed effects models is large, the mixed effect model with subject ID as a random effect appears to account for substantially more variance in the reaction time than the fixed effects model even after accounting for complexity.

# Reflection (1 point)

What other random effects could be controlled for in this data set? 
> We should control for the individual word items (e.g. stimulus ID for the words) to see whether there are any effect driven by particular words. 

**DUE:** 5pm EST, March 25, 2021

**IMPORTANT** Did you collaborate with anyone on this assignment? If so, list their names here. 
> Urszula Oszczapinska

> Austin Luor